 # Extended Versions of Standardized FMA and SFMA

In [1]:
from functions_SFMA import *

## Set parameters and load the initial datset and W matrix

In [2]:
num_N = 6 # number of rows of a matrix 
nrep_it = 0.01 # number of iterations of SFMA 
nrep_it_ext = 0.01 # extra number of iterations of SFMA 
n_samp = 1 # number of sampling of SFMA 
ratio = 0.4 # ratio for creating a subsampling dataset
num_W = 2 # index of a target W matrix

In [3]:
n_bit = num_N*num_K # number of spins (qubits)
n_in = n_bit # size of the initial dataset 
k_FM = int(n_bit/2)-1 # hyperparameter for a FM model 

val_dim = f'd{n_bit}'
val_standard = "standard"
val_k = str(k_FM)
val_anneal = 'SA'

n_it = pow(n_bit,2) # unit of the number of iterations of SFMA
n_it_tot = int(n_it*nrep_it) # number of the iterations
n_tot = n_in + 1 + n_it_tot # size of the dataset with n_it_tot = 2*n_it^2  
val_samp = f'samp{n_samp}'
val_n_tot = str(n_tot)

# Load the target W matrix, the mean and std values, and the objective values. 

val_num_W = f'W_{num_W}'
folder_data =  f'final_data_file_{n_bit}'
init_file = f'{folder_data}/initial_Data_{val_num_W}_{val_dim}_{val_standard}_{val_k}_{val_anneal}.bz2' # file containing the inital dataset
with open(init_file, 'rb') as inD:
    initD = pickle.load(inD)

M_W = initD['Matrix_W'] # target W matrix
mean_W = initD['mean'] # mean
std_W = initD['std'] # std
minBB_1st_W = initD['minBB_1st'] # 1st optimal objective (output) value
minBB_2nd_W = initD['minBB_2nd'] # 2nd optimal objective value
minBB_mid2_W = initD['minBB_mid2'] # mid2 optimal value

# Load the results of xs and ys with ratio=0.4 and n_tot = 2*n_it^2 + 1.   
result_file = f'{folder_data}/result_{val_num_W }_{val_dim}_{val_k}_{val_anneal}_{val_samp}_{val_n_tot}_{val_standard}.bz2' # file containing the result
with open(result_file, 'rb') as res_fl:
    res_D = pickle.load(res_fl)
###
xs_full_result = res_D['ensmeble_xs_full'] # xs via FMA
ys_full_result = res_D['ensmeble_ys_full'] # ys via FMA
xs_sub_result = res_D['ensmeble_xs_sub'] # xs via SFMA
ys_sub_result = res_D['ensmeble_ys_sub'] # ys via SFMA

## Perform FMA and SFMA and save the results

In [4]:
n_it_tot_ext = int(n_it*nrep_it_ext) # number of extra iterations of SFMA 
n_tot_ext = n_tot + n_it_tot_ext # size of the final extended dataset 
val_n_tot_ext = str(n_tot_ext)
opt_1st = np.array([minBB_1st_W]*n_tot_ext) # array of minBB_1st_W
opt_2nd = np.array([minBB_2nd_W]*n_tot_ext) # array of minBB_2nd_W
opt_mid2 = np.array([minBB_mid2_W]*n_tot_ext) # array of minBB_mid2_W 
opt_1st_rounded = np.round(opt_1st, n_round) # rounded opt_1st 
opt_2nd_rounded = np.round(opt_2nd, n_round) # rounded opt_2nd 
opt_mid2_rounded = np.round(opt_mid2, n_round) # rounded opt_mid2 
#### Setup
xs_int_full_W, ys_int_full_W = xs_full_result, ys_full_result # use the dataset with the size n_tot as an inital dataset
xs_int_sub_W, ys_int_sub_W = xs_sub_result, ys_sub_result # use the dataset with the size n_tot as an inital dataset
################################################################ FMA
list_samp_xs_full =[] # list for samples of inputs (xs)
list_samp_ys_full =[] # list for samples of outputs (xs)
for samp in range(n_samp):
    xs, ys = xs_int_full_W[samp], ys_int_full_W[samp] # prepare the inital dataset
    FMmodel_set, xs, ys = train_full_standard_SA(x=xs, y=ys, W=M_W, n_N=num_N, n_K=num_K, Mean=mean_W, Stand_Deviate=std_W, seed_Torch=samp, num_it=n_it_tot_ext) # FMA via SA
    list_samp_xs_full.append(xs) # append the final xs to list_samp_xs_full 
    list_samp_ys_full.append(ys) # append the final xs to list_samp_ys_full 
################################################################ SFMA
list_samp_xs_sub =[] # list for samples of xs
list_samp_ys_sub =[] # list for samples of ys
for samp in range(n_samp):
    xs, ys = xs_int_sub_W[samp], ys_int_sub_W[samp] # prepare the inital dataset
    FMmodel_set, xs, ys = train_sub_standard_SA(x=xs, y=ys, W=M_W, n_N=num_N, n_K=num_K, Mean=mean_W, Stand_Deviate=std_W, seed_Torch=samp, num_it=n_it_tot_ext, R=ratio, num_seed=seed_sub) # SFMA via SA
    list_samp_xs_sub.append(xs) # append the final xs to list_samp_xs_sub
    list_samp_ys_sub.append(ys) # append the final xs to list_samp_ys_sub
################################################################ statistical calculation
ensmeble_xs_full = np.array(list_samp_xs_full) # n_samp samples of the final xs
ensmeble_ys_full = np.array(list_samp_ys_full) # n_samp samples of the final ys
ensmeble_ys_min_full = output_sample_min(ensmeble_ys_full) # n_samp samples of the final minimum ys (ys_min)
ys_mean_full = np.mean(ensmeble_ys_min_full, axis=0) # averaged ys_min along the vertical direction
ys_mean_full_rounded = np.round(ys_mean_full,n_round) # rounded ys_mean_full
rate_full = success_rate(ensmeble_ys_full, n_in, n_tot_ext, minBB_1st_W, n_samp) # success rate
rate_full_rounded = success_rate_rounded(ensmeble_ys_full, n_in, n_tot_ext, n_round, minBB_1st_W, n_samp) # rounded success rate
####
ensmeble_xs_sub = np.array(list_samp_xs_sub) # n_samp samples of the final xs
ensmeble_ys_sub = np.array(list_samp_ys_sub) # n_samp samples of the final ys
ensmeble_ys_min_sub = output_sample_min(ensmeble_ys_sub) # n_samp samples of the final ys_min
ys_mean_sub = np.mean(ensmeble_ys_min_sub, axis=0) # averaged ys_min along the vertical direction
ys_mean_sub_rounded = np.round(ys_mean_sub,n_round) # rounded ys_mean_sub 
rate_sub = success_rate(ensmeble_ys_sub, n_in, n_tot_ext, minBB_1st_W, n_samp) # success rate  
rate_sub_rounded = success_rate_rounded(ensmeble_ys_sub, n_in, n_tot_ext, n_round, minBB_1st_W, n_samp) # rounded success rate

################################################################ Save the results.
dictionary = { 'ensmeble_xs_full': ensmeble_xs_full, 'ensmeble_ys_full': ensmeble_ys_full,'ensmeble_ys_min_full': ensmeble_ys_min_full, 
                  'ys_mean_full': ys_mean_full, 'ys_mean_full_rounded': ys_mean_full_rounded, 
                  'rate_full': rate_full,'rate_full_rounded': rate_full_rounded,
              'ensmeble_xs_sub': ensmeble_xs_sub, 'ensmeble_ys_sub': ensmeble_ys_sub,'ensmeble_ys_min_sub': ensmeble_ys_min_sub,
        'ys_mean_sub': ys_mean_sub, 'ys_mean_sub_rounded': ys_mean_sub_rounded,
                  'rate_sub': rate_sub, 'rate_sub_rounded': rate_sub_rounded,
              'opt_1st': opt_1st, 'opt_2nd': opt_2nd, 'opt_mid2': opt_mid2, 
                  'opt_1st_rounded': opt_1st_rounded, 'opt_2nd_rounded': opt_2nd_rounded, 'opt_mid2_rounded': opt_mid2_rounded
             }
file = f'{folder_data}/extended_result_{val_num_W}_{val_dim}_{val_k}_{val_anneal}_{val_samp}_{val_n_tot}_{val_n_tot_ext}_{val_standard}.bz2'

with open(file, 'wb') as fl:
    pickle.dump(dictionary, fl)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 27.44it/s]
